In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os

base_path = "/content/drive/MyDrive/Data"
print("Folders inside:", os.listdir(base_path))

Folders inside: ['pins_amber heard', 'pins_Mark Zuckerberg', 'pins_scarlett johansson', 'pins_Rebecca Ferguson', 'pins_Tom Cruise', 'pins_Zendaya', 'pins_Wentworth Miller', 'dataset.xlsx', 'pins_tom ellis', 'pins_Zac Efron', 'pins_Tuppence Middleton', 'pins_Tom Hiddleston', 'pins_Ursula Corbero', 'pins_Tom Holland', 'pins_Penn Badgley', 'pins_Rami Malek', 'pins_Tom Hardy', 'pins_Sophie Turner', 'pins_Robert De Niro', 'pins_Shakira Isabel Mebarak', 'pins_Sarah Wayne Callies', 'pins_Stephen Amell', 'pins_Richard Harmon', 'pins_Rihanna', 'pins_Morena Baccarin', 'pins_Robert Downey Jr', 'pins_Selena Gomez', 'pins_Keanu Reeves', 'pins_melissa fumero', 'pins_Natalie Portman', 'pins_Miley Cyrus', 'pins_Mark Ruffalo', 'pins_Maisie Williams', 'pins_Alex Lawther', 'pins_Neil Patrick Harris', 'pins_Megan Fox', 'pins_Marie Avgeropoulos', 'pins_Nadia Hilker', 'pins_Pedro Alonso', 'pins_Morgan Freeman', 'pins_Millie Bobby Brown', 'pins_Natalie Dormer', 'pins_Andy Samberg', 'pins_Katharine Mcphee', '

In [5]:
DATASET_DIR = "/content/drive/MyDrive/Data"

In [6]:
print("Top-level identities:", os.listdir(DATASET_DIR)[:10])

Top-level identities: ['pins_amber heard', 'pins_Mark Zuckerberg', 'pins_scarlett johansson', 'pins_Rebecca Ferguson', 'pins_Tom Cruise', 'pins_Zendaya', 'pins_Wentworth Miller', 'dataset.xlsx', 'pins_tom ellis', 'pins_Zac Efron']


In [7]:
import pandas as pd

excel_path = "/content/drive/MyDrive/Data/dataset.xlsx"
df = pd.read_excel(excel_path)

print(df.head())
print(df.columns)

                                     ID                 Name    Status  Age
0  bf723908-8899-11f0-bbe4-0242ac1c000c         Adriana Lima    denied   42
1  bf723a48-8899-11f0-bbe4-0242ac1c000c         Alex Lawther    denied   42
2  bf723aca-8899-11f0-bbe4-0242ac1c000c   Alexandra Daddario  approved   51
3  bf723b42-8899-11f0-bbe4-0242ac1c000c         Alvaro Morte    denied   40
4  bf723bba-8899-11f0-bbe4-0242ac1c000c  alycia dabnem carey    denied   43
Index(['ID', 'Name', 'Status', 'Age'], dtype='object')


In [8]:
!pip -q install openpyxl rapidfuzz

import os, re, shutil, random, pandas as pd
from rapidfuzz import process, fuzz
random.seed(42)

# <-- عدّلي هذا لمسار مجلد الصور (كل مجلد = شخص)
BASE_DIR = "/content/drive/MyDrive/Data"

# <-- عدّلي هذا لمسار ملف الإكسل
EXCEL_PATH = "/content/drive/MyDrive/Data/dataset.xlsx"

df_meta = pd.read_excel(EXCEL_PATH, engine="openpyxl")
df_meta.columns = [str(c).strip().lower() for c in df_meta.columns]
print("Columns:", df_meta.columns.tolist())
assert {"id","name","status","age"}.issubset(df_meta.columns), "ملف الإكسل لازم يحتوي الأعمدة: id, name, status, age"

# طبّعي الأعمدة المهمة
df_meta["name"] = df_meta["name"].astype(str).str.strip()
df_meta["status"] = df_meta["status"].astype(str).str.strip().str.lower()

# المجلدات الموجودة فعليًا على القرص
folders_on_disk = [d for d in os.listdir(BASE_DIR) if os.path.isdir(os.path.join(BASE_DIR, d))]
print("Folders on disk (sample):", folders_on_disk[:10], " | total:", len(folders_on_disk))


Columns: ['id', 'name', 'status', 'age']
Folders on disk (sample): ['pins_amber heard', 'pins_Mark Zuckerberg', 'pins_scarlett johansson', 'pins_Rebecca Ferguson', 'pins_Tom Cruise', 'pins_Zendaya', 'pins_Wentworth Miller', 'pins_tom ellis', 'pins_Zac Efron', 'pins_Tuppence Middleton']  | total: 105


In [9]:
def strip_prefix(name: str):
    # يشيل "pins_" أو أي بادئة منتهية بـ "_" مرة وحدة
    return re.sub(r'^[A-Za-z]+_', '', name).strip()

def slugify(s: str):
    # توحيد: حروف صغيرة، حذف الرموز، مسافات/شرطات -> _
    s = s.lower().strip()
    s = s.replace("&","and")
    s = re.sub(r"[\'’`]", "", s)
    s = re.sub(r"[_\s\-]+", "_", s)
    s = re.sub(r"[^a-z0-9_]", "", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s

# حضّري سلاجات من الإكسل
df_meta["name_slug"] = df_meta["name"].map(slugify)

# حضّري سلاج للفولدرات (بعد إزالة البادئة)
folder_core   = {f: strip_prefix(f) for f in folders_on_disk}        # "Adriana Lima"
folder_slug   = {f: slugify(folder_core[f]) for f in folders_on_disk}# "adriana_lima"


In [10]:
# خرايط lookup من الإكسل
name_to_idx = {df_meta.loc[i, "name_slug"]: i for i in range(len(df_meta))}

matched = {}
unmatched = []

for f in folders_on_disk:
    fslug = folder_slug[f]
    if fslug in name_to_idx:
        matched[f] = name_to_idx[fslug]
    else:
        unmatched.append(f)

print(f"Matched {len(matched)} / {len(folders_on_disk)}")
print("Unmatched sample:", unmatched[:8])


Matched 105 / 105
Unmatched sample: []


In [98]:
import os, re, random, shutil, pandas as pd
from sklearn.model_selection import train_test_split
random.seed(42)

# مساراتك
BASE_DIR   = "/content/drive/MyDrive/Data"  # يحوي مجلدات: pins_Adriana Lima ...
EXCEL_PATH = "/content/drive/MyDrive/Data/dataset.xlsx"     # فيه id, name, status, age
OUT_ROOT   = "/content/data"  # سيُنشأ train/ و val/
os.makedirs(OUT_ROOT, exist_ok=True)

def is_img(f): return f.lower().endswith(('.jpg','.jpeg','.png','.webp'))

def strip_prefix(name: str):
    # يشيل أي بادئة منتهية بـ "_" مثل pins_
    return re.sub(r'^[A-Za-z]+_', '', name).strip()

# 1) اقرأ الإكسل وحددي status
dfm = pd.read_excel(EXCEL_PATH)
dfm.columns = [str(c).strip().lower() for c in dfm.columns]
assert {"id","name","status","age"}.issubset(dfm.columns)
dfm["name"]   = dfm["name"].astype(str).str.strip()
dfm["status"] = dfm["status"].astype(str).str.strip().str.lower()

# 2) ابنِ جدول الصور: label = اسم المجلد بعد إزالة pins_
rows = []
folders = [d for d in os.listdir(BASE_DIR) if os.path.isdir(os.path.join(BASE_DIR, d))]
for folder in folders:
    identity = strip_prefix(folder)  # مثال: pins_Adriana Lima -> Adriana Lima
    fpath = os.path.join(BASE_DIR, folder)
    # ملاحظة: هون ربطنا الاسم الداخلي بالـidentity مباشرة (اسم المجلد بدون prefix)
    for fn in os.listdir(fpath):
        if not is_img(fn): continue
        rows.append({
            "path": os.path.join(fpath, fn),
            "identity": identity,
            "filename": fn
        })
df = pd.DataFrame(rows)

# 3) فلترة حسب Status من الإكسل (اختياري لكن أنصح فيه)
# نفترض إن عمود name في الإكسل يطابق identity (بدون pins_). لو فيه اختلافات طفيفة، خبريني أضيف لك تطبيع/مطابقة.
approved_names = set(dfm.loc[dfm["status"].isin(["denied","approved"]), "name"])
if len(approved_names) > 0:
    before = len(df)
    df = df[df["identity"].isin(approved_names)].copy()
    print(f"Status filter: {before} -> {len(df)} images kept (approved only)")

# تأكيد حد أدنى للصور لكل هوية (مع بياناتك غالبًا الجميع > 100)
df = df.groupby("identity").filter(lambda g: len(g) >= 20).reset_index(drop=True)
print("Identities:", df["identity"].nunique())

# 4) Split 80/20 (Stratified)
train_df, val_df = train_test_split(
    df, test_size=0.2, stratify=df["identity"], random_state=42
)

# ====== (اختياري) السيناريو B: Sampling على الـTrain فقط ======
DO_SAMPLING = False     # ← غيّريها إلى True لو بدك سرعة أعلى
MAX_PER_ID_TRAIN = 120  # حد أعلى لكل هوية في Train

if DO_SAMPLING:
    def cap_train(group):
        return group.sample(n=min(MAX_PER_ID_TRAIN, len(group)), random_state=42)
    train_df = train_df.groupby("identity", group_keys=False).apply(cap_train).reset_index(drop=True)

print("Train images:", len(train_df), "| Val images:", len(val_df))

# 5) نسخ لهيكل التدريب
TRAIN_DIR = os.path.join(OUT_ROOT, "train")
VAL_DIR   = os.path.join(OUT_ROOT, "val")
for p in (TRAIN_DIR, VAL_DIR): os.makedirs(p, exist_ok=True)

def copy_split(split_df, root):
    for _, r in split_df.iterrows():
        ident = r["identity"]
        src   = r["path"]
        dst_dir = os.path.join(root, ident)
        os.makedirs(dst_dir, exist_ok=True)
        shutil.copy2(src, os.path.join(dst_dir, os.path.basename(src)))

copy_split(train_df, TRAIN_DIR)
copy_split(val_df,   VAL_DIR)

print("Train IDs:", len([d for d in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, d))]))
print("Val IDs:",   len([d for d in os.listdir(VAL_DIR)   if os.path.isdir(os.path.join(VAL_DIR, d))]))


Status filter: 15478 -> 15478 images kept (approved only)
Identities: 105
Train images: 12382 | Val images: 3096


KeyboardInterrupt: 

In [11]:
import os, random, shutil, cv2
random.seed(42)

# عدّلي هذا لمسار المجلد الذي يحتوي على كل فولدرات الأشخاص:
SRC_ROOT = "/content/drive/MyDrive/Data"

# مكان إخراج العينة المقسّمة:
DST_TRAIN = "/content/sampled/train"
DST_VAL   = "/content/sampled/val"
os.makedirs(DST_TRAIN, exist_ok=True)
os.makedirs(DST_VAL,   exist_ok=True)

# كم صورة بدك من كل فولدر إجمالاً (Train+Val)؟
SAMPLE_PER_FOLDER = 120   # غيّريه لـ 80/100/150 حسب وقتك

# نسبة التحقق:
VAL_RATIO = 0.2  # 20% Val


In [12]:
import os

# إذا فولدر Data موجود داخل "Colab Notebooks":
SRC_ROOT = "/content/drive/MyDrive/Data"

# (بديل) إذا Data في MyDrive مباشرة:
# SRC_ROOT = "/content/drive/MyDrive/Data"

print("SRC_ROOT =", SRC_ROOT)
print("Sample folders:", os.listdir(SRC_ROOT)[:6])


SRC_ROOT = /content/drive/MyDrive/Data
Sample folders: ['pins_amber heard', 'pins_Mark Zuckerberg', 'pins_scarlett johansson', 'pins_Rebecca Ferguson', 'pins_Tom Cruise', 'pins_Zendaya']


In [13]:
def is_img(f): return f.lower().endswith(('.jpg','.jpeg','.png','.webp'))

def count_images(root):
    c=0
    for r,_,fs in os.walk(root):
        c += sum(is_img(f) for f in fs)
    return c

train_ids = len([d for d in os.listdir(DST_TRAIN) if os.path.isdir(os.path.join(DST_TRAIN,d))])
val_ids   = len([d for d in os.listdir(DST_VAL)   if os.path.isdir(os.path.join(DST_VAL,d))])

print("Train IDs:", train_ids, "| Val IDs:", val_ids)
print("Train images:", count_images(DST_TRAIN))
print("Val images:",   count_images(DST_VAL))


Train IDs: 105 | Val IDs: 105
Train images: 6717
Val images: 1679


In [15]:
# استخدمي إحدى المجموعتين:
TRAIN_DIR = "/content/sampled/train"
VAL_DIR   = "/content/sampled/val"

# أو الداتا الكاملة:
# TRAIN_DIR = "/content/full/train"
# VAL_DIR   = "/content/full/val"

import os
print("Train:", os.path.exists(TRAIN_DIR), "| Val:", os.path.exists(VAL_DIR))


Train: True | Val: True


In [16]:
!pip -q install facenet-pytorch torchvision pillow tqdm opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.

In [25]:
!pip install torchvision


In [32]:
import torch, torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import cv2, numpy as np, os
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# كاشف وجوه بسيط من OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def detect_and_crop_face(img_bgr, margin=0.2, out_size=(224,224)):
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)
    if len(faces)==0:
        # فشل الكشف → استخدم الصورة كاملة (آخر حل)
        face = cv2.resize(img_bgr, out_size, interpolation=cv2.INTER_AREA)
        return Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
    # خذ أكبر وجه
    x,y,w,h = sorted(faces, key=lambda b: b[2]*b[3], reverse=True)[0]
    dx, dy = int(margin*w), int(margin*h)
    x1=max(0,x-dx); y1=max(0,y-dy); x2=min(img_bgr.shape[1], x+w+dx); y2=min(img_bgr.shape[0], y+h+dy)
    crop = img_bgr[y1:y2, x1:x2]
    face = cv2.resize(crop, out_size, interpolation=cv2.INTER_AREA)
    return Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))

# تحولات ResNet
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])


ImportError: cannot import name 'is_directory' from 'PIL._util' (/usr/local/lib/python3.12/dist-packages/PIL/_util.py)

In [140]:
facebank, counts = {}, {}
used_train, skipped_train = 0, 0

for ident, files in tqdm(list(list_images(TRAIN_DIR)), desc="Build facebank"):
    use = files[:20] if len(files) > 20 else files  # 5–20 صورة/هوية تكفي كبداية
    try:
        reps = DeepFace.represent(
            img_path=use,
            model_name=MODEL,
            detector_backend=BACKEND,
            enforce_detection=False,
            align=True
        )
    except Exception as e:
        print("Error on", ident, "->", e)
        continue

    vecs = []
    for r in reps:
        emb = r.get("embedding", None)
        if emb is None:
            skipped_train += 1
            continue
        vecs.append(l2norm(emb))
        used_train += 1

    if not vecs:
        continue
    M = np.stack(vecs, axis=0)
    centroid = l2norm(M.mean(axis=0))
    facebank[ident] = centroid
    counts[ident]   = len(vecs)

idents = sorted(facebank.keys())
centroids = np.stack([facebank[i] for i in idents], axis=0) if len(idents) else None
print(f"Identities enrolled: {len(idents)} | train images used: {used_train} | skipped: {skipped_train}")


Build facebank:   1%|          | 1/105 [00:01<02:43,  1.57s/it]

Error on pins_Adriana Lima -> 'KerasHistory' object has no attribute 'layer'


Build facebank:   2%|▏         | 2/105 [00:02<02:25,  1.42s/it]

Error on pins_Alex Lawther -> 'KerasHistory' object has no attribute 'layer'


Build facebank:   3%|▎         | 3/105 [00:04<02:13,  1.31s/it]

...

Error on pins_melissa fumero -> 'KerasHistory' object has no attribute 'layer'


Build facebank:  99%|█████████▉| 104/105 [02:16<00:01,  1.19s/it]

Error on pins_scarlett johansson -> 'KerasHistory' object has no attribute 'layer'


Build facebank: 100%|██████████| 105/105 [02:17<00:00,  1.31s/it]

Error on pins_tom ellis -> 'KerasHistory' object has no attribute 'layer'
Identities enrolled: 0 | train images used: 0 | skipped: 0


In [141]:
total, correct = 0, 0
scores_true, scores_false = [], []
used_val, skipped_val = 0, 0

for ident, files in tqdm(list(list_images(VAL_DIR)), desc="Eval on val"):
    if ident not in facebank:
        continue
    reps = DeepFace.represent(
        img_path=files,
        model_name=MODEL,
        detector_backend=BACKEND,
        enforce_detection=False,
        align=True
    )
    for r in reps:
        emb = r.get("embedding", None)
        if emb is None:
            skipped_val += 1
            continue
        emb = l2norm(emb)
        # تصنيف 1:N
        sims = centroids @ emb
        pred_idx = int(np.argmax(sims))
        pred = idents[pred_idx]
        total   += 1
        correct += int(pred == ident)
        used_val += 1

        # درجات التحقق (موجب مقابل أقرب سالب)
        scores_true.append(cosine(emb, facebank[ident]))
        if len(idents) > 1:
            other = np.delete(centroids, pred_idx, axis=0)
            scores_false.append(float(np.max(other @ emb)))
        else:
            scores_false.append(0.0)

acc = correct / max(total, 1)
print(f"\n=== RESULTS ===")
print(f"Top-1 Identification Accuracy: {acc:.4f}  ({correct}/{total})")
print(f"Val images used: {used_val} | skipped (no embedding): {skipped_val}")

# اختيار أفضل عتبة للتحقق (maximize F1)
best_f1, best_th = -1, 0.35
ths = np.linspace(0.2, 0.8, 61)
y_true  = np.array([1]*len(scores_true) + [0]*len(scores_false))
y_score = np.array(scores_true + scores_false, dtype="float32")

for th in ths:
    y_pred = (y_score >= th).astype(int)
    tp = int(((y_pred==1)&(y_true==1)).sum())
    fp = int(((y_pred==1)&(y_true==0)).sum())
    fn = int(((y_pred==0)&(y_true==1)).sum())
    prec = tp / max(tp+fp,1)
    rec  = tp / max(tp+fn,1)
    f1 = 2*prec*rec / max(prec+rec,1e-8)
    if f1 > best_f1:
        best_f1, best_th = f1, th

print(f"Best Verification F1: {best_f1:.4f} at Threshold = {best_th:.3f}")


Eval on val: 100%|██████████| 105/105 [00:00<00:00, 209515.66it/s]


=== RESULTS ===
Top-1 Identification Accuracy: 0.0000  (0/0)
Val images used: 0 | skipped (no embedding): 0
Best Verification F1: 0.0000 at Threshold = 0.200


In [136]:
import time, hashlib
from deepface import DeepFace

# تحميل المخرجات (لو فتحتِ نوتبوك جديد)
# centroids = np.load("/content/facebank.npy")
# with open("/content/idents.json") as f: idents = json.load(f)
# with open("/content/config.json") as f: cfg = json.load(f)
# MODEL, BACKEND, THRESHOLD = cfg["model"], cfg["backend"], float(cfg["threshold"])

def cosine(a,b):
    return float(np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b)+1e-8))

def verify(username, image_path, threshold=THRESHOLD):
    t0 = time.time()
    if username not in idents:
        return {"decision":"DENIED","reason":"user_not_enrolled"}
    # استخراج embedding للصورة
    rep = DeepFace.represent(
        img_path=image_path, model_name=MODEL, detector_backend=BACKEND,
        enforce_detection=False, align=True
    )
    if not rep or "embedding" not in rep[0]:
        return {"decision":"DENIED","reason":"no_embedding"}

    emb = np.array(rep[0]["embedding"], dtype="float32")
    emb = emb/(np.linalg.norm(emb)+1e-8)

    # إيجاد سكّور المستخدم الصحيح
    idx = idents.index(username)
    user_vec = centroids[idx]
    score = cosine(emb, user_vec)

    decision = "GRANTED" if score >= threshold else "DENIED"
    return {
        "ts": time.strftime("%Y-%m-%d %H:%M:%S"),
        "username_claim": username,
        "decision": decision,
        "score": round(score,4),
        "threshold": round(threshold,3),
        "latency_ms": int((time.time()-t0)*1000),
        "model": f"DeepFace-{MODEL}",
        "backend": BACKEND,
        "image_md5": hashlib.md5(open(image_path,'rb').read()).hexdigest()
    }

# تجربة سريعة (عدّلي المسار):
test_img = "/content/sampled/val/pins_Alexandra Daddario"
verify("pins_Alexandra Daddario", test_img)


{'decision': 'DENIED', 'reason': 'user_not_enrolled'}